This script performs visualization and post-processing of YOLO bounding box annotations 
to address imperfections in the labels generated from SAM body label generation. The goal 
is to clean up and refine the dataset before training a YOLO model.

Key steps:
1. **Setup:**
   - Defines directories for training and validation images and labels.
   - Creates output directories for visualizations and cleaned labels.

2. **Visualization:**
   - Converts YOLO format bounding box annotations 
     (<class-id> <x-center> <y-center> <width> <height>) to pixel coordinates.
   - Identifies bounding boxes that:
       - Cover more than 8% of the image area.
       - Exceed 50% of the image width or height.
   - Draws these bounding boxes on the corresponding images for inspection.
   - Saves the visualized images to dedicated directories.

3. **Post-Processing Labels:**
   - Removes bounding box labels that do not meet the area or size thresholds.
   - Rewrites label files with only valid bounding boxes for both training and validation datasets.

4. **Output:**
   - Visualized images with problematic bounding boxes are saved for review.
   - Cleaned and refined YOLO label files are generated.

This ensures that the dataset used for YOLO training is free of invalid or overly small bounding boxes, 
improving model performance and training reliability.


In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Directories
img_dir = '../data/final-dataset/train/images'
label_dir = '../data/final-dataset/train/labels'

# Define paths
merged_data_dir = os.path.join("..", 'data', 'final-dataset')

train_img_dir = os.path.join(merged_data_dir, 'train', 'images')
train_label_dir = os.path.join(merged_data_dir, 'train', 'labels')
val_img_dir = os.path.join(merged_data_dir, 'val', 'images')
val_label_dir = os.path.join(merged_data_dir, 'val', 'labels')

# Output directories for visualizations
train_output_dir = os.path.join(merged_data_dir, 'problemvis', 'train')
val_output_dir = os.path.join(merged_data_dir, 'problemvis', 'val')

# Create output directories if they don't exist
os.makedirs(train_output_dir, exist_ok=True)
os.makedirs(val_output_dir, exist_ok=True)


In [ ]:



# Function to convert YOLO format to bounding box coordinates
def yolo_to_bbox(yolo_bbox, img_width, img_height):
    class_id, x_center, y_center, width, height = map(float, yolo_bbox)
    
    x_min = int((x_center - width / 2) * img_width)
    x_max = int((x_center + width / 2) * img_width)
    y_min = int((y_center - height / 2) * img_height)
    y_max = int((y_center + height / 2) * img_height)
    
    return class_id, x_min, y_min, x_max, y_max

# Function to plot and save the image with bounding boxes
def plot_image_with_boxes(image_path, labels, output_dir, image_filename):
    image = cv2.imread(image_path)
    img_height, img_width, _ = image.shape
    
    for yolo_bbox in labels:
        class_id, x_min, y_min, x_max, y_max = yolo_to_bbox(yolo_bbox, img_width, img_height)
        
        # Draw the bounding box
        # only draw if the box area is larger than 0.08 of the image area
        if (x_max - x_min) * (y_max - y_min) > 0.08 * img_height * img_width or (x_max - x_min) > 0.5 * img_width or (y_max - y_min) > 0.5 * img_height:
            cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 0, 255), 2)
            cv2.putText(image, str(int(class_id)), (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)
    
    # Save the visualized image
    output_image_path = os.path.join(output_dir, image_filename)
    cv2.imwrite(output_image_path, image)

# Function to process images and labels
def process_images_and_labels(img_dir, label_dir, output_dir):
    image_files = sorted([f for f in os.listdir(img_dir) if f.endswith('.jpg')])
    
    for idx, image_file in enumerate(image_files):
        # Get the corresponding label file
        label_file = image_file.replace('.jpg', '.txt')
        label_path = os.path.join(label_dir, label_file)
        image_path = os.path.join(img_dir, image_file)
        
        # Read the label file
        with open(label_path, 'r') as file:
            labels = [line.strip().split() for line in file.readlines()]
        
        # Check if the box area is larger than 0.08 of the image area
        img_height, img_width, _ = cv2.imread(image_path).shape
        wide_or_tall_boxes = False
        for yolo_bbox in labels:
            class_id, x_min, y_min, x_max, y_max = yolo_to_bbox(yolo_bbox, img_width, img_height)
            if ((x_max - x_min) * (y_max - y_min) > 0.08 * img_height * img_width) or ((x_max - x_min) > 0.5 * img_width) or ((y_max - y_min) > 0.5 * img_height):
                wide_or_tall_boxes = True
                break

        
        if wide_or_tall_boxes:
            
            # Plot current image
            plot_image_with_boxes(image_path, labels, output_dir, 'current_' + image_file)

            
            # Plot next image if available
            if idx < len(image_files) - 1:
                next_image_file = image_files[idx + 1]
                next_image_path = os.path.join(img_dir, next_image_file)
                next_label_file = next_image_file.replace('.jpg', '.txt')
                next_label_path = os.path.join(label_dir, next_label_file)
                with open(next_label_path, 'r') as file:
                    next_labels = [line.strip().split() for line in file.readlines()]
                plot_image_with_boxes(next_image_path, next_labels, output_dir, 'next_' + next_image_file)

            
            

# Process train images and labels
process_images_and_labels(train_img_dir, train_label_dir, train_output_dir)

# Process val images and labels
process_images_and_labels(val_img_dir, val_label_dir, val_output_dir)

print("Visualization complete.")


In [ ]:

# Function to convert YOLO format to bounding box coordinates
def yolo_to_bbox(yolo_bbox, img_width, img_height):
    class_id, x_center, y_center, width, height = map(float, yolo_bbox)
    
    x_min = int((x_center - width / 2) * img_width)
    x_max = int((x_center + width / 2) * img_width)
    y_min = int((y_center - height / 2) * img_height)
    y_max = int((y_center + height / 2) * img_height)
    
    return class_id, x_min, y_min, x_max, y_max

# Function to process images and labels
def process_images_and_labels(img_dir, label_dir):
    image_files = sorted([f for f in os.listdir(img_dir) if f.endswith('.jpg')])
    
    for image_file in image_files:
        # Get the corresponding label file
        label_file = image_file.replace('.jpg', '.txt')
        label_path = os.path.join(label_dir, label_file)
        image_path = os.path.join(img_dir, image_file)
        
        # Read the label file
        with open(label_path, 'r') as file:
            labels = [line.strip().split() for line in file.readlines()]
        
        # Check if the box area is larger than 0.08 of the image area
        img_height, img_width, _ = cv2.imread(image_path).shape
        new_labels = []
        has_large_box = False
        
        for yolo_bbox in labels:
            class_id, x_min, y_min, x_max, y_max = yolo_to_bbox(yolo_bbox, img_width, img_height)
            
            #if (x_max - x_min) * (y_max - y_min) > 0.08 * img_height * img_width:
            if ((x_max - x_min) * (y_max - y_min) > 0.08 * img_height * img_width) or ((x_max - x_min) > 0.5 * img_width) or ((y_max - y_min) > 0.5 * img_height):

                has_large_box = True
            else:
                new_labels.append(yolo_bbox)

        # Only rewrite label file if there is at least one large box
        if has_large_box:
            with open(label_path, 'w') as file:
                for label in new_labels:
                    file.write(' '.join(label) + '\n')

# Process train images and labels
process_images_and_labels(train_img_dir, train_label_dir)

# Process val images and labels
process_images_and_labels(val_img_dir, val_label_dir)

print("Label cleanup complete.")
